## 📦 1. Importar Librerías

pandas, numpy, sklearn para manipulación de datos y feature engineering.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
print('✅ Librerías cargadas')

✅ Librerías cargadas


## 📂 2. Cargar Datos

Carga de train, test, features y stores desde data/01_raw/

In [2]:
RAW_PATH = '../data/01_raw/'
PROCESSED_PATH = '../data/02_processed/'

train = pd.read_csv(RAW_PATH + 'train.csv')
test = pd.read_csv(RAW_PATH + 'test.csv')
features = pd.read_csv(RAW_PATH + 'features.csv')
stores = pd.read_csv(RAW_PATH + 'stores.csv')

print(f'Train: {train.shape}, Test: {test.shape}, Features: {features.shape}, Stores: {stores.shape}')

train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])
features['Date'] = pd.to_datetime(features['Date'])
print('✅ Datos cargados')

Train: (421570, 5), Test: (115064, 4), Features: (8190, 12), Stores: (45, 3)


✅ Datos cargados


## 🔗 3. Merge Datasets

Union de train/test con features (por Store, Date, IsHoliday) y stores (por Store).

In [3]:
df_train = train.merge(features, on=['Store','Date','IsHoliday'], how='left')
df_train = df_train.merge(stores, on='Store', how='left')
df_test = test.merge(features, on=['Store','Date','IsHoliday'], how='left')
df_test = df_test.merge(stores, on='Store', how='left')
print(f'Train: {df_train.shape}, Test: {df_test.shape}')

Train: (421570, 16), Test: (115064, 15)

## 🧹 4. Rellenar MarkDowns con 0

Los valores faltantes en promociones se asumen como 0 (sin descuento).

In [4]:
markdown_cols = ['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']
for col in markdown_cols:
    df_train[col].fillna(0, inplace=True)
    df_test[col].fillna(0, inplace=True)
print('✅ MarkDowns rellenados con 0')

✅ MarkDowns rellenados con 0


## ⚡ 5. Features Temporales

16 features: Year, Month, Week, Quarter, DayOfWeek, sin/cos encoding, Trend.

In [5]:
def create_temporal_features(df):
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week
    df['Quarter'] = df['Date'].dt.quarter
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['DayOfYear'] = df['Date'].dt.dayofyear
    df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
    df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)
    df['IsQuarterStart'] = df['Date'].dt.is_quarter_start.astype(int)
    df['IsQuarterEnd'] = df['Date'].dt.is_quarter_end.astype(int)
    df['WeekOfMonth'] = (df['Date'].dt.day - 1) // 7 + 1
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    df['Week_sin'] = np.sin(2 * np.pi * df['Week'] / 52)
    df['Week_cos'] = np.cos(2 * np.pi * df['Week'] / 52)
    df['Trend'] = (df['Date'] - df['Date'].min()).dt.days // 7
    return df

df_train = create_temporal_features(df_train)
df_test = create_temporal_features(df_test)
print('✅ Features temporales creadas')

✅ Features temporales creadas


## 🎄 6. Features de Festivos

8 features: banderas para cada festivo + días hasta/desde festivo + pre/post-festivo.

In [6]:
def create_holiday_features(df):
    holidays = {
        'SuperBowl': ['2010-02-12','2011-02-11','2012-02-10'],
        'LaborDay': ['2010-09-10','2011-09-09','2012-09-07'],
        'Thanksgiving': ['2010-11-26','2011-11-25','2012-11-23'],
        'Christmas': ['2010-12-31','2011-12-30','2012-12-28']
    }
    for name, dates in holidays.items():
        dates = [pd.to_datetime(d) for d in dates]
        df[f'Is{name}'] = df['Date'].isin(dates).astype(int)
    
    all_holidays = []
    for dates in holidays.values():
        all_holidays.extend([pd.to_datetime(d) for d in dates])
    all_holidays = sorted(set(all_holidays))
    
    df['DaysToNextHoliday'] = df['Date'].apply(lambda x: min([(h-x).days for h in all_holidays if h>=x], default=365))
    df['DaysFromLastHoliday'] = df['Date'].apply(lambda x: min([(x-h).days for h in all_holidays if h<=x], default=365))
    df['IsPreHoliday'] = (df['DaysToNextHoliday'] <= 7).astype(int)
    df['IsPostHoliday'] = (df['DaysFromLastHoliday'] <= 7).astype(int)
    return df

df_train = create_holiday_features(df_train)
df_test = create_holiday_features(df_test)
print('✅ Features de festivos creadas')

✅ Features de festivos creadas


## 📊 7. Features de Lag (Solo Train)

4 features: ventas de 1, 2, 3, 4 semanas previas (solo para entrenamiento).

In [7]:
df_train = df_train.sort_values(['Store','Dept','Date'])
for lag in [1,2,3,4]:
    df_train[f'Weekly_Sales_Lag{lag}'] = df_train.groupby(['Store','Dept'])['Weekly_Sales'].shift(lag)
print('✅ Lag features creadas')

✅ Lag features creadas


## 📈 8. Rolling Features (Solo Train)

12 features: promedio, std, min, max móviles para ventanas de 4, 8, 12 semanas.

In [8]:
for window in [4,8,12]:
    df_train[f'Weekly_Sales_RollingMean{window}'] = df_train.groupby(['Store','Dept'])['Weekly_Sales'].transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
    df_train[f'Weekly_Sales_RollingStd{window}'] = df_train.groupby(['Store','Dept'])['Weekly_Sales'].transform(lambda x: x.shift(1).rolling(window, min_periods=1).std())
    df_train[f'Weekly_Sales_RollingMin{window}'] = df_train.groupby(['Store','Dept'])['Weekly_Sales'].transform(lambda x: x.shift(1).rolling(window, min_periods=1).min())
    df_train[f'Weekly_Sales_RollingMax{window}'] = df_train.groupby(['Store','Dept'])['Weekly_Sales'].transform(lambda x: x.shift(1).rolling(window, min_periods=1).max())
print('✅ Rolling features creadas')

✅ Rolling features creadas


## 🏬 9. Features Agregadas Store-Dept

5 features: estadísticas históricas (mean, std, min, max, median) por tienda y departamento.

In [9]:
agg_stats = df_train.groupby(['Store','Dept'])['Weekly_Sales'].agg([('StoreDept_Mean','mean'),('StoreDept_Std','std'),('StoreDept_Min','min'),('StoreDept_Max','max'),('StoreDept_Median','median')]).reset_index()
df_train = df_train.merge(agg_stats, on=['Store','Dept'], how='left')
df_test = df_test.merge(agg_stats, on=['Store','Dept'], how='left')
print('✅ Features agregadas creadas')

✅ Features agregadas creadas


## 🔀 10. Features de Interacción

7 features: combinaciones como Type_Holiday, Total_MarkDown, Temp_Month, Econ_Index.

In [10]:
def create_interaction_features(df):
    df['Type_Holiday'] = df['Type'].astype(str) + '_' + df['IsHoliday'].astype(str)
    df['Store_Dept'] = df['Store'].astype(str) + '_' + df['Dept'].astype(str)
    df['Temp_Month'] = df['Temperature'] * df['Month']
    df['Size_Holiday'] = df['Size'] * df['IsHoliday']
    df['Total_MarkDown'] = df[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].sum(axis=1)
    df['Count_MarkDown'] = (df[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']] > 0).sum(axis=1)
    df['Econ_Index'] = df['Unemployment'] * df['CPI']
    return df

df_train = create_interaction_features(df_train)
df_test = create_interaction_features(df_test)
print('✅ Features de interacción creadas')

✅ Features de interacción creadas


## 🔤 11. Encoding Categóricas

LabelEncoding para Type, Type_Holiday, Store_Dept (convertir texto a números).

In [11]:
for col in ['Type','Type_Holiday','Store_Dept']:
    le = LabelEncoder()
    df_train[col+'_Encoded'] = le.fit_transform(df_train[col].astype(str))
    df_test[col+'_Encoded'] = df_test[col].astype(str).apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
print('✅ Encoding completado')

✅ Encoding completado


## 🧹 12. Limpieza de NaNs

Relleno de valores faltantes en lag/rolling features con promedio por Store-Dept.

In [12]:
lag_rolling_cols = [col for col in df_train.columns if 'Lag' in col or 'Rolling' in col]
for col in lag_rolling_cols:
    df_train[col] = df_train.groupby(['Store','Dept'])[col].transform(lambda x: x.fillna(x.mean()))
    df_train[col].fillna(0, inplace=True)

if 'StoreDept_Std' in df_train.columns:
    df_train['StoreDept_Std'].fillna(0, inplace=True)
    df_test['StoreDept_Std'].fillna(0, inplace=True)

print(f'NaNs restantes train: {df_train.isnull().sum().sum()}')
print(f'NaNs restantes test: {df_test.isnull().sum().sum()}')

NaNs restantes train: 0
NaNs restantes test: 114630


## ✂️ 13. Train/Validation Split

División temporal: 85% para entrenamiento, 15% para validación.

In [13]:
df_train = df_train.sort_values('Date')
train_dates = sorted(df_train['Date'].unique())
split_idx = int(len(train_dates) * 0.85)
split_date = train_dates[split_idx]

X_train_full = df_train[df_train['Date'] < split_date].copy()
X_val = df_train[df_train['Date'] >= split_date].copy()

print(f'Split date: {split_date}')
print(f'Train: {X_train_full.shape[0]:,} | Validation: {X_val.shape[0]:,}')

Split date: 2012-06-01 00:00:00
Train: 356,489 | Validation: 65,081


## 📊 14. Preparar X, y para Modelado

Separación de features (X) y target (y), eliminando columnas no necesarias.

In [14]:
features_to_drop = ['Date','Weekly_Sales','Type','Type_Holiday','Store_Dept']
y_train = X_train_full['Weekly_Sales']
y_val = X_val['Weekly_Sales']
X_train = X_train_full.drop(columns=features_to_drop, errors='ignore')
X_val_features = X_val.drop(columns=features_to_drop, errors='ignore')

print(f'X_train: {X_train.shape} | X_val: {X_val_features.shape}')
print(f'y_train: {y_train.shape} | y_val: {y_val.shape}')
print(f'Total features: {X_train.shape[1]}')

X_train: (356489, 66) | X_val: (65081, 66)
y_train: (356489,) | y_val: (65081,)
Total features: 66


## 📦 15. Exportar Datos Procesados

Exportación a data/02_processed/: train/val/test procesados y X/y separados.

In [15]:
import os
os.makedirs(PROCESSED_PATH, exist_ok=True)

X_train_full.to_csv(PROCESSED_PATH + 'train_processed.csv', index=False)
X_val.to_csv(PROCESSED_PATH + 'val_processed.csv', index=False)
df_test.to_csv(PROCESSED_PATH + 'test_processed.csv', index=False)
X_train.to_csv(PROCESSED_PATH + 'X_train.csv', index=False)
X_val_features.to_csv(PROCESSED_PATH + 'X_val.csv', index=False)
y_train.to_csv(PROCESSED_PATH + 'y_train.csv', index=False, header=['Weekly_Sales'])
y_val.to_csv(PROCESSED_PATH + 'y_val.csv', index=False, header=['Weekly_Sales'])

print(f'✅ Datos exportados a: {PROCESSED_PATH}')
for f in os.listdir(PROCESSED_PATH):
    if f.endswith('.csv'):
        size_mb = os.path.getsize(PROCESSED_PATH+f)/(1024**2)
        print(f'  {f} ({size_mb:.2f} MB)')

✅ Datos exportados a: ../data/02_processed/
  test_processed.csv (36.59 MB)
  train_processed.csv (173.97 MB)
  val_processed.csv (32.79 MB)
  X_train.csv (162.21 MB)
  X_val.csv (30.64 MB)
  y_train.csv (3.07 MB)
  y_val.csv (0.56 MB)


## ✅ Resumen Final

**FEATURES CREADAS:**
- Temporales: 16 (Year, Month, Week, Quarter, etc.)
- Festivos: 8 (IsSuperBowl, IsLaborDay, etc.)
- Lag: 4 (1-4 semanas atrás)
- Rolling: 12 (mean, std, min, max para 4/8/12 semanas)
- Agregadas: 5 (StoreDept statistics)
- Interacción: 7 (Type_Holiday, Total_MarkDown, etc.)
- Encoding: 3 (Type_Encoded, etc.)

**TOTAL: ~{X_train.shape[1]} features**

**PRÓXIMO:** 03_modelado_dataset.ipynb